# Computational Theory 

In [1780]:
# for numerical data and methods 
import numpy as np 

## Problem 1: Binary Words and Operations


The `numpy.int32()` constructor ([see official documentation] https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.int32) ensures that values are stored and treated as 32-bit signed integers in NumPy.

### Parity Function
The `parity` function calculates the bitwise XOR of three 32-bit integers, as defined in the SHA-1 algorithm. For each bit position, it returns a result of 1 when an odd number of the input values is set to 1, and 0 otherwise. This is implemented as `x ^ y ^ z`.

In [1781]:
def parity(x,y,z):
    """Calculate the parity of three 32-bit integers"""
    # np.uint32 ensures inputs are treated as 32-bit unsigned integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to keep only the lower 32 bits, handles negative numbers correctly and simulates 32-bit overflow behavior.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)

    # Calculate the bitwise XOR of the three 32-bit values to get the parity.
    parity_output = np.uint32(x ^ y ^ z)
    # Return the result.
    return parity_output

### Test Parity Function
This section tests the `parity` function with various inputs to verify its correctness.

In [1782]:
# Test parity function 
# Test 1: All zeros
print("TEST 1: parity(0, 0, 0)")
test_result = parity(0, 0, 0)
expected_result = 0
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 2: One non-zero input
print("\nTEST 2: parity(1, 0, 0)")
test_result = parity(1, 0, 0)
expected_result = 1
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 3: Two non-zero inputs
print("\nTEST 3: parity(1, 1, 0)")
test_result = parity(1, 1, 0)
expected_result = 0
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 4: Negative numbers
print("\nTEST 4: parity(-1, -2, -3)")
test_result = parity(-1, -2, -3)
expected_result = 4294967292
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 5: Mixed inputs
print("\nTEST 5: parity(2, -4, 5)")
test_result = parity(2, -4, 5)
expected_result = 4294967291
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 6: All ones
print("\nTEST 6: parity(1, 1, 1)")
test_result = parity(1, 1, 1)
expected_result = 1
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 7: Three negative inputs
print("\nTEST 7: parity(-1, -1, -1)")
test_result = parity(-1, -1, -1)
expected_result = 4294967295
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

TEST 1: parity(0, 0, 0)
Result :   0x0 
Expected :  0x0 
Correct :  True

TEST 2: parity(1, 0, 0)
Result :   0x1 
Expected :  0x1 
Correct :  True

TEST 3: parity(1, 1, 0)
Result :   0x0 
Expected :  0x0 
Correct :  True

TEST 4: parity(-1, -2, -3)
Result :   0xfffffffc 
Expected :  0xfffffffc 
Correct :  True

TEST 5: parity(2, -4, 5)
Result :   0xfffffffb 
Expected :  0xfffffffb 
Correct :  True

TEST 6: parity(1, 1, 1)
Result :   0x1 
Expected :  0x1 
Correct :  True

TEST 7: parity(-1, -1, -1)
Result :   0xffffffff 
Expected :  0xffffffff 
Correct :  True


### Ch (choose) Function
The `choose` function, used in the SHA-224 and SHA-256 algorithms, calculates a conditional selection among three 32-bit integers. For each bit position, it returns the bit from y if the bit in x is 1, or the bit from z if the bit in x is 0. This is implemented as `(x & y) ^(~x & z)`.

In [1783]:
def ch(x, y, z):
    """Calculate the choose function for three 32-bit integers"""
    # np.uint32 ensures inputs are treated as 32-bit unsigned integers as per SHA-1 specification.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to keep only the lower 32 bits, handles negative numbers correctly and simulates 32-bit overflow behavior.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)
    
    # Bitwise AND of x and y.
    bitwise_and = (x & y)
    # Bitwise complement of x, then AND with z.
    bitwise_complement_and = (~x) & z
    # Calculate the bitwise XOR of the two results and cast to 32-bit integer.
    # This selects bits from y or z based on the value of x.
    ch_output = np.uint32(bitwise_and ^ bitwise_complement_and)
    
    # Return the result.
    return ch_output 

### Test Ch Function
This section tests the `ch` function with various inputs to verify its correctness.

In [1784]:
# Test ch function 
# Test 1: All zeros
print("TEST 1: ch(0, 0, 0)")
test_result = ch(0, 0, 0)
expected_result = 0
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 2: All ones
print("\nTEST 2: ch(1, 1, 1)")
test_result = ch(1, 1, 1)
expected_result = 1
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 3: Only z is non-zero
print("\nTEST 3: ch(0, 0, 1)")
test_result = ch(0, 0, 1)
expected_result = 1
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 4: Only z is zero
print("\nTEST 4: ch(1, 1, 0)")
test_result = ch(1, 1, 0)
expected_result = 1
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 5: All negative ones
print("\nTEST 5: ch(-1, -1, -1)")
test_result = ch(-1, -1, -1)
expected_result = 4294967295
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 6: Negative and positive numbers 
print("\nTEST 6: ch(-1, 0, 1)")
test_result = ch(-1, 0, 1)
expected_result = 0
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)


TEST 1: ch(0, 0, 0)
Result :   0x0 
Expected :  0x0 
Correct :  True

TEST 2: ch(1, 1, 1)
Result :   0x1 
Expected :  0x1 
Correct :  True

TEST 3: ch(0, 0, 1)
Result :   0x1 
Expected :  0x1 
Correct :  True

TEST 4: ch(1, 1, 0)
Result :   0x1 
Expected :  0x1 
Correct :  True

TEST 5: ch(-1, -1, -1)
Result :   0xffffffff 
Expected :  0xffffffff 
Correct :  True

TEST 6: ch(-1, 0, 1)
Result :   0x0 
Expected :  0x0 
Correct :  True


### Majority Function
The `maj` function, used in the SHA-224 and SHA-256 algorithms, calculates the majority value among three 32-bit integers. For each bit position, it returns 1 if at least two of the three bits among x, y, and z are 1, and 0 otherwise. This is implemented as `(x & y) ^ (x & z) ^ (y & z)`.

In [1785]:
def maj(x, y, z):
    """Calculate the majority value of three 32-bit integers."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)
    y = np.uint32(y & 0xFFFFFFFF)
    z = np.uint32(z & 0xFFFFFFFF)

    # Bitwise AND of x and y.
    bitwise_and_x_y = (x & y)   
    # Bitwise AND of x and z.
    bitwise_and_x_z = (x & z) 
    # Bitwise AND of y and z.   
    bitwise_and_y_z = (y & z)   

    # Calculate the bitwise XOR of the three results to get the majority value and cast to a 32-bit integer.
    maj_output = np.uint32(bitwise_and_x_y ^ bitwise_and_x_z ^ bitwise_and_y_z)
    # Return the result.
    return maj_output 

### Test Maj Function
This section tests the `maj` function with various inputs to verify its correctness.

In [1786]:
# Test maj function
# Test 1: All zeros
print("TEST 1: maj(0, 0, 0)")
test_result = maj(0, 0, 0)
expected_result = 0
print("Result   : ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect  : ", test_result == expected_result)

# Test 2: All ones
print("\nTEST 2: maj(1, 1, 1)")
test_result = maj(1, 1, 1)
expected_result = 1
print("Result   : ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 3: One non-zero input
print("\nTEST 3: maj(1, 0, 0)")
test_result = maj(1, 0, 0)
expected_result = 0
print("Result   : ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect  : ", test_result == expected_result)

# Test 4: Two non-zero inputs
print("\nTEST 4: maj(2, 2, 0)")
test_result = maj(2, 2, 0)
expected_result = 2
print("Result   : ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect  : ", test_result == expected_result)

# Test 5: All different numbers
print("\nTEST 5: maj(3, 5, 7)")
test_result = maj(3, 5, 7)
expected_result = 7
print("Result   : ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect  : ", test_result == expected_result)

TEST 1: maj(0, 0, 0)
Result   :  0x0 
Expected :  0x0 
Correct  :  True

TEST 2: maj(1, 1, 1)
Result   :  0x1 
Expected :  0x1 
Correct :  True

TEST 3: maj(1, 0, 0)
Result   :  0x0 
Expected :  0x0 
Correct  :  True

TEST 4: maj(2, 2, 0)
Result   :  0x2 
Expected :  0x2 
Correct  :  True

TEST 5: maj(3, 5, 7)
Result   :  0x7 
Expected :  0x7 
Correct  :  True


In [1787]:
def rotr(x,n):
    """Rotate right operation for 32-bit words: A helper function for the sigma functions."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x = np.uint32(x)
    # 32 bit word size
    return np.uint32(((x >> n) | (x << np.uint32(32 - n))) & np.uint32(0xFFFFFFFF))


### Sigma0 Function
In the SHA-224 and SHA-256 algorithms, the `sigma0` function applies a series of bitwise operations to a single 32-bit integer.  It computes the bitwise XOR of three rotated versions of the input value — one by two bits to the right, one by thirteen bits, and one by twenty-two bits. This is implemented as `ROTR^2(x) ^ ROTR^13(x) ^ ROTR^22(x)`.

In [1788]:
def sigma_upper_0(x):
    """Calculate the Sigma0 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)
    
    # Compute the bitwise XOR of three right rotated versions of the input value(x).
    return (rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


### Test Sigma0 Function
This section tests the `Sigma0` function with various inputs to verify its correctness.

In [1789]:
# Test 1: A number other than 0 or 1
print("\nTEST 1: sigma_upper_0(5)")
test_result = sigma_upper_0(5)
expected_result = 1076368385
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 2: A zero input
print("\nTEST 2: sigma_upper_0(0)")
test_result = sigma_upper_0(0)
expected_result = 0
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 3: Input of one
print("\nTEST 3: sigma_upper_0(1)")
test_result = sigma_upper_0(1)
expected_result = 1074267136
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 4: A negative input
print("\nTEST 4: sigma_upper_0(-1)")
test_result = sigma_upper_0(-1)
expected_result = 4294967295
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)


TEST 1: sigma_upper_0(5)
Result :   0x40281401 
Expected :  0x40281401 
Correct :  True

TEST 2: sigma_upper_0(0)
Result :   0x0 
Expected :  0x0 
Correct :  True

TEST 3: sigma_upper_0(1)
Result :   0x40080400 
Expected :  0x40080400 
Correct :  True

TEST 4: sigma_upper_0(-1)
Result :   0xffffffff 
Expected :  0xffffffff 
Correct :  True


### Sigma1 Function 
In the SHA-224 and SHA-256 algorithms, the `Sigma1` function applies a series of bitwise operations to a single 32-bit integer.  It computes the bitwise XOR of three rotated versions of the input value — one by six bits to the right, one by 11 bits, and one by twenty-five bits. This is implemented as `ROTR^6(x) ^ ROTR^11(x) ^ ROTR^25(x)`.

In [1790]:
def sigma_upper_1(x):
    """Calculate the Sigma1 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of three right-rotated versions of the input value(x).
    return (rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

### Test Sigma1 Function
This section tests the `Sigma1` function with various inputs to verify its correctness.

In [1791]:
# Test 1: Input other than 0 or 1
print("\nTEST 1: sigma_upper_1(5)")
test_result = sigma_upper_1(5)
expected_result = 346030720  
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 2: A zero input
print("\nTEST 2: sigma_upper_1(0)")
test_result = sigma_upper_1(0)
expected_result = 0 
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 3: An input of one
print("\nTEST 3: sigma_upper_1(1)")
test_result = sigma_upper_1(1)
expected_result = 69206144 
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)

# Test 4: A negative input
print("\nTEST 4: sigma_upper_1(-1)")
test_result = sigma_upper_1(-1)
expected_result = 4294967295  
print("Result :  ", hex(test_result), "\nExpected : ", hex(expected_result), "\nCorrect : ", test_result == expected_result)



TEST 1: sigma_upper_1(5)
Result :   0x14a00280 
Expected :  0x14a00280 
Correct :  True

TEST 2: sigma_upper_1(0)
Result :   0x0 
Expected :  0x0 
Correct :  True

TEST 3: sigma_upper_1(1)
Result :   0x4200080 
Expected :  0x4200080 
Correct :  True

TEST 4: sigma_upper_1(-1)
Result :   0xffffffff 
Expected :  0xffffffff 
Correct :  True


### The sigma0 Function 
In the SHA-224 and SHA-256 algorithms, the `sigma0` function applies a series of bitwise operations to a single 32-bit integer. It computes the bitwise XOR of two right-rotated versions of the input value—one rotated right by 7 bits, one rotated right by 18 bits—and the value shifted right by 3 bits. This is implemented as `ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)`.

In [1792]:
def sigma_lower_0(x):
    """Calculate the sigma0 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of two right-rotated versions of the input value and the right-shifted value.
    return (rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

### Test sigma0 Function 
This section tests the `sigma0` function with various inputs to verify its correctness.

In [1793]:
# Test 1: Input other than 0 or 1
print("\nTEST 1: sigma_lower_0(5)")
test_result = sigma_lower_0(5)
expected_result = 167854080  
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 2: A zero input
print("\nTEST 2: sigma_lower_0(0)")
test_result = sigma_lower_0(0)
expected_result = 0  
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 3: An input of one
print("\nTEST 3: sigma_lower_0(1)")
test_result = sigma_lower_0(1)
expected_result = 33570816 
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 4: A negative input
print("\nTEST 4: sigma_lower_0(-1)")
test_result = sigma_lower_0(-1)
expected_result = 536870911
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)


TEST 1: sigma_lower_0(5)
Result :   167854080 
Expected :  167854080 
Correct :  True

TEST 2: sigma_lower_0(0)
Result :   0 
Expected :  0 
Correct :  True

TEST 3: sigma_lower_0(1)
Result :   33570816 
Expected :  33570816 
Correct :  True

TEST 4: sigma_lower_0(-1)
Result :   536870911 
Expected :  536870911 
Correct :  True


### The sigma0 Function 
In the SHA-224 and SHA-256 algorithms, the `sigma0` function applies a series of bitwise operations to a single 32-bit integer. It computes the bitwise XOR of two right-rotated versions of the input value—one rotated right by 7 bits, one rotated right by 18 bits—and the value shifted right by 3 bits. This is implemented as `ROTR^7(x) ^ ROTR^18(x) ^ SHR^3(x)`.


In [1794]:
def sigma_lower_0(x):
    """Calculate the sigma0 value for a 32-bit integer."""
    # The np.uint32() constructor ensures inputs are treated as unsigned 32-bit integers. 
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    # Mask with 0xFFFFFFFF to ensure only the lower 32 bits are kept to handle negative inputs correctly.
    x = np.uint32(x & 0xFFFFFFFF)

    # Compute the bitwise XOR of two right-rotated versions of the input value and the right-shifted value.
    return (rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

### Test sigma0 Function 
This section tests the `sigma0` function with various inputs to verify its correctness.


In [1795]:
# Test 1: Input other than 0 or 1
print("\nTEST 1: sigma_lower_0(5)")
test_result = sigma_lower_0(5)
expected_result = 167854080  
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 2: A zero input
print("\nTEST 2: sigma_lower_0(0)")
test_result = sigma_lower_0(0)
expected_result = 0  
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 3: An input of one
print("\nTEST 3: sigma_lower_0(1)")
test_result = sigma_lower_0(1)
expected_result = 33570816 
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)

# Test 4: A negative input
print("\nTEST 4: sigma_lower_0(-1)")
test_result = sigma_lower_0(-1)
expected_result = 536870911
print("Result :  ", test_result, "\nExpected : ", expected_result, "\nCorrect : ", test_result == expected_result)


TEST 1: sigma_lower_0(5)
Result :   167854080 
Expected :  167854080 
Correct :  True

TEST 2: sigma_lower_0(0)
Result :   0 
Expected :  0 
Correct :  True

TEST 3: sigma_lower_0(1)
Result :   33570816 
Expected :  33570816 
Correct :  True

TEST 4: sigma_lower_0(-1)
Result :   536870911 
Expected :  536870911 
Correct :  True


## Problem 2: Fractional Parts of Cube Roots 


In [1796]:
# Write a function called primes(n) that generates the first n prime numbers.

# Use the function to calculate the cube root of the first 64 primes.

# For each cube root, extract the first thirty-two bits of the fractional part.

# Display the result in hexadecimal.

# Test the results against what is in the Secure Hash Standard.

## Problem 3: Padding


In [1797]:
# Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. 

## Problem 4: Hashes


In [1798]:
# Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

## Problem 5: Passwords


In [1799]:
# The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm. As strings, they were encoded using UTF-8. Determine the passwords and explain how you found them. 

## End